# **MLP(Multi-Layer Perceptron) 구현하기**

**데이터 로드 및 전처리**

먼저, 가장 널리 사용되는 딥러닝 프레임워크(framework) 중 하나인 텐서플로(TensorFlow)를 임포트하고, 

라이브러리로부터 MNIST데이터를 로드하고 전처리를 수행한다.

클래스 레이블을 원핫벡터(one-hot vector)로 변환하는 과정은 사용하고자 하는 프레임워크와 모델의 요구에 맞추어 선택적으로 거치는 과정이다.

In [2]:
import os


# Import tensorflow packages
import tensorflow as tf
import numpy as np

# MNIST 데이터 로드 및 전처리
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0    # 데이터 정규화

print(np.shape(x_test))
print(np.shape(y_test))
print(np.unique(y_test), '\n') # 클래스 레이블 확인

# 클래스 레이블 one-hot vector 변환
y_train_1hot = tf.one_hot(y_train, 10)
y_test_1hot = tf.one_hot(y_test, 10)
print('one-hot vector 변환 후 테스트 데이터 클래스 레이블 차원 수 : ', np.shape(y_test_1hot))
print('클래스 ', y_test[0], '의 one-hot vector : ', y_test_1hot[0,:])

(10000, 28, 28)
(10000,)
[0 1 2 3 4 5 6 7 8 9] 

one-hot vector 변환 후 테스트 데이터 클래스 레이블 차원 수 :  (10000, 10)
클래스  7 의 one-hot vector :  tf.Tensor([0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(10,), dtype=float32)


**다층 퍼셉트론 모델 생성**

모델 선언을 통해 2개의 은닉층으로 이루어진 다층 퍼셉트론(MLP)을 구성한다.

분류할 클래스의 수가 3개 이상이고, one-hot vector로 레이블이 주어질 경우 categorical crossentropy를 사용한다.

In [4]:
# 다층 퍼셉트론 모델 생성
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28)),    # Flatten 28x28 and set input layer with 756 nodes
  tf.keras.layers.Dense(128, activation = 'relu'),    # 128개 노드를 가진 첫 번째 은닉층 (relu activation function)
  # activations : 'sigmoid','tanh', 'relu'
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(128, activation = 'relu'),    # 128개 노드를 가진 두 번째 은닉층 (relu activation function)
  tf.keras.layers.Dropout(0.2), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, activation = 'softmax')   # 10개(MNIST의 클래스 수) 출력 노드를 가진 출력층 (activation function: 'softmax')
])

# 생성된 모델의 구조 확인하기
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                      

**학습 전략 설정**

최적화 알고리즘 및 손실함수 등을 설정한다.

SGD(Stochastic Gradient Descent), Adam(Adaptive Moment Estimation), RMSprop 등의 최적화 기법들은 각자 장단점이 있으며, 

풀고자 하는 문제, 신경망 모델과 데이터의 특성에 따라 최적의 효율을 가지는 최적화 기법이 다르므로

여러번의 실험을 통해 적절한 최적화 기법을 선택해야 한다. 

In [7]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.0)             
#opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
#opt = tf.keras.optimizers.RMSprop(learning_rate = 0.001, momentum = 0.0)

model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
#model.compile(loss = 'mse', optimizer = opt, metrics=['accuracy'])  

#model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**모델 학습 및 성능 측정**

앞서 구성한 다층 퍼셉트론 모델을 학습 데이터로 훈련시키고, 테스트 데이터에 대한 성능을 측정한다.

fit 함수에서 파라미터로 설정 가능한 'batch_size'는 데이터를 학습할 때 한 번에 몇 개의 데이터로 학습할 것인지를 의미하며,

'epoch'는 전체 데이터를 몇 번 반복 학습할 것인지를 나타낸다.

여기에서는 time 라이브러리를 통해 학습에 걸린 시간을 측정해 본다.

'time.time()'함수는 프로그램이 실행된 이후 현재 시점의 시간을 리턴한다.


In [11]:
import time

# 모델 학습
start_time = time.time()
model.fit (x_train, y_train_1hot, batch_size = 200, epochs = 5)        # Training with "categorical crossentropy"
print("모델 학습 완료. 학습 시간 : {}\n".format(time.time() - start_time))

# 테스트 데이터로 학습된 모델의 성능 측정
print('테스트 데이터에 대한 분류 성능')
test_loss, test_acc = model.evaluate(x_test,  y_test_1hot, batch_size = 100, verbose = 2)

Epoch 1/5
300/300 [==============================] - 2s 7ms/step - loss: 0.1524 - accuracy: 0.9545
Epoch 2/5
300/300 [==============================] - 2s 7ms/step - loss: 0.1484 - accuracy: 0.9561
Epoch 3/5
300/300 [==============================] - 2s 7ms/step - loss: 0.1456 - accuracy: 0.9567
Epoch 4/5
300/300 [==============================] - 2s 7ms/step - loss: 0.1446 - accuracy: 0.9570
Epoch 5/5
300/300 [==============================] - 2s 7ms/step - loss: 0.1374 - accuracy: 0.9583
모델 학습 완료. 학습 시간 : 10.834697723388672

테스트 데이터에 대한 분류 성능
100/100 - 0s - loss: 0.1010 - accuracy: 0.9685 - 234ms/epoch - 2ms/step


위 코드의 실행 결과창을 살펴보면, 총 5번의 epoch 동안 학습하였으며, 각 epoch마다 '300/300'이라는 숫자를 확인할 수 있다.

이는 'iteration'을 나타내는 것으로, 한 번에 batch_size만큼 입력하여 전체 학습 데이터를 모두 학습하는 데 걸리는 반복 횟수를 의미한다.

즉, 200의 batch_size만큼 300번을 반복해야 총 60,000개의 학습 데이터를 모두 한 번씩 학습할 수 있다는 것이다.

이처럼 batch_size를 나누어 한 번에 입력되는 학습 데이터의 개수를 제한하는 것은 메모리와 학습 속도 때문이다.

한 번에 모든 데이터를 입력하여 학습하는 경우 메모리 부족 문제로  학습 속도가 느려지거나 불가능하게 된다.

이러한 문제는 데이터의 개수가 많을수록, 데이터의 차원이 클수록 더 심해진다.

